# Natural Language Processing Final Assignment

## Real - Fake News Classification: A Comparison of Natural Language Models for Classification

In [28]:
# Import Libraries
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from sklearn.pipeline import make_pipeline
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier



In [12]:
# Load Data

df_raw = pd.read_csv("news_dataset.csv")

ParserError: Error tokenizing data. C error: EOF inside string starting at row 34730

In [30]:
df_raw = pd.read_json("Sarcasm_Headlines_Dataset.json", lines=True)

In [13]:
# Load Data

# Use the 'python' engine which is more flexible for parsing errors
df_raw = pd.read_csv("news_dataset.csv", engine='python')

In [31]:
df_raw.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


### Preprocessing Function

In [32]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
def preprocess_text(text):

    text = text.str.lower()

    # this remove links and twitter handels, and emails etc
    text = text.str.replace(r'http\S+|www\S+|https\S+', '', regex=True)
    text = text.str.replace(r'\S+@\S+', '', regex=True)
    text = text.str.replace(r'@\w+', '', regex=True)

    text = text.str.replace(f'[{string.punctuation}]', '', regex=True)

    text = text.str.replace(r'\d+', '', regex=True)

    text = text.str.replace(r'\s+', ' ', regex=True).str.strip()

    text_tokens = text.apply(nltk.word_tokenize)

    stop_words = set(stopwords.words('english'))
    text_tokens = text_tokens.apply(lambda tokens: [w for w in tokens if w not in stop_words])

    lemmatizer = WordNetLemmatizer()
    text_tokens = text_tokens.apply(lambda tokens: [lemmatizer.lemmatize(w) for w in tokens])

    processed_text = text_tokens.apply(lambda tokens: ' '.join(tokens))

    return processed_text


In [34]:
df = df_raw.copy()

In [20]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
# Download the punkt_tab resource
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [21]:
df["title_preprocessed"] = preprocess_text(df["title"])
df["text_preprocessed"] = preprocess_text(df["text"])

In [22]:
df["features"] = df["title_preprocessed"] + " " + df["text_preprocessed"]

In [35]:
df["headline_clean"] = preprocess_text(df["headline"])

### Train - Test Split

In [23]:
X = df["features"]
y = df["label"]

In [36]:
X = df["headline_clean"]
y = df["is_sarcastic"]

In [37]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, stratify=y, random_state=7)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=7)



X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [38]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(18696,)
(4006,)
(4007,)


In [39]:
# ===== NEW: Shuffled Labels Experiment =====
print("\n=== Shuffled Labels Experiment ===")

# Shuffle the training labels to destroy any real relationship
y_train_shuffled = y_train.sample(frac=1.0, random_state=42).reset_index(drop=True)

# Sanity check: make sure labels are actually shuffled
assert not all(y_train_shuffled == y_train), "Shuffling failed: labels match original"

# Create pipeline with TF-IDF + Logistic Regression
shuffled_pipeline = make_pipeline(
    TfidfVectorizer(stop_words='english', ngram_range=(1, 1)),
    LogisticRegression(penalty='l2', C=0.1, max_iter=1000)
)

print("Fitting model on shuffled labels...")
shuffled_pipeline.fit(X_train, y_train_shuffled)

print("Predicting on validation set with correct labels...")
y_val_pred_shuffled = shuffled_pipeline.predict(X_val)

print("Evaluating model trained on shuffled labels:")
def evaluate_models(y_true, y_pred_dict, label=None):
    metrics = {
        'Model': [],
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F1-score': []
    }
    for model_name, y_pred in y_pred_dict.items():
        print(f"\n===== {model_name}: {label if label else ''} =====")
        print("Classification Report:")
        print(classification_report(y_true, y_pred))

        metrics['Model'].append(model_name)
        metrics['Accuracy'].append(accuracy_score(y_true, y_pred))

        unique_labels = np.unique(y_true)
        if len(unique_labels) == 2:
            metrics['Precision'].append(precision_score(y_true, y_pred, average='binary', pos_label=unique_labels[1]))
            metrics['Recall'].append(recall_score(y_true, y_pred, average='binary', pos_label=unique_labels[1]))
            metrics['F1-score'].append(f1_score(y_true, y_pred, average='binary', pos_label=unique_labels[1]))
        else:
            metrics['Precision'].append(precision_score(y_true, y_pred, average='weighted'))
            metrics['Recall'].append(recall_score(y_true, y_pred, average='weighted'))
            metrics['F1-score'].append(f1_score(y_true, y_pred, average='weighted'))

    return pd.DataFrame(metrics)

evaluate_models(y_val, {"Shuffled Labels Model": y_val_pred_shuffled}, label="Validation Set")
# ===== END NEW: Shuffled Labels Experiment =====


=== Shuffled Labels Experiment ===
Fitting model on shuffled labels...
Predicting on validation set with correct labels...
Evaluating model trained on shuffled labels:

===== Shuffled Labels Model: Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.56      1.00      0.72      2248
           1       0.43      0.00      0.00      1758

    accuracy                           0.56      4006
   macro avg       0.49      0.50      0.36      4006
weighted avg       0.50      0.56      0.40      4006



,Model,Accuracy,Precision,Recall,F1-score
0,Shuffled Labels Model,0.560909,0.428571,0.001706,0.003399


## Model Fitting

### Define functions for efficient model evaluation and fitting

In [40]:
def evaluate_models(y_true, y_pred_dict, label=None):
    """
    Evaluate and compare multiple models' predictions.
    Parameters:
    y_true : array-like
        True labels.
    y_pred_dict : dict
        Dictionary where keys are model names and values are predicted labels.
    label : str, optional
        Description of the evaluation set (e.g., "Validation Set").
    Returns:
    pd.DataFrame containing accuracy, precision, recall, and F1 for each model.
    """

    metrics = {
        'Model': [],
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F1-score': []
    }

    for model_name, y_pred in y_pred_dict.items():
        print(f"\n===== {model_name}: {label if label else ''} =====")
        print("Classification Report:")
        print(classification_report(y_true, y_pred))

        metrics['Model'].append(model_name)
        metrics['Accuracy'].append(accuracy_score(y_true, y_pred))

        unique_labels = np.unique(y_true)
        if len(unique_labels) == 2:
            metrics['Precision'].append(precision_score(y_true, y_pred, average='binary', pos_label=unique_labels[1]))
            metrics['Recall'].append(recall_score(y_true, y_pred, average='binary', pos_label=unique_labels[1]))
            metrics['F1-score'].append(f1_score(y_true, y_pred, average='binary', pos_label=unique_labels[1]))
        else:
            metrics['Precision'].append(precision_score(y_true, y_pred, average='weighted'))
            metrics['Recall'].append(recall_score(y_true, y_pred, average='weighted'))
            metrics['F1-score'].append(f1_score(y_true, y_pred, average='weighted'))

    return pd.DataFrame(metrics)


In [41]:
def fit_pipeline_predict_evaluate(
    model_class,
    model_name,
    X_train,
    y_train,
    X_val,
    y_val,
    vectorizer_type='tfidf',
    ngram_range=(1,1)
):
    if vectorizer_type == 'tfidf':
        vectorizer = TfidfVectorizer(stop_words='english', ngram_range=ngram_range)
    elif vectorizer_type == 'bow':
        vectorizer = CountVectorizer(stop_words='english', ngram_range=ngram_range)
    else:
        raise ValueError("vectorizer_type must be either 'tfidf' or 'bow'")

    pipeline = make_pipeline(vectorizer, model_class)

    print(f"Fitting {model_name}...")
    pipeline.fit(X_train, y_train)

    print(f"Predicting on training set...")
    y_pred_train = pipeline.predict(X_train)

    print(f"Predicting on validation set...")
    y_pred_val = pipeline.predict(X_val)

    print(f"Evaluating {model_name} on training set...")
    train_metrics = evaluate_models(y_train, {model_name: y_pred_train}, label="Train Set")

    print(f"Evaluating {model_name} on validation set...")
    val_metrics = evaluate_models(y_val, {model_name: y_pred_val}, label="Validation Set")

    train_metrics["Dataset"] = "Train"
    val_metrics["Dataset"] = "Validation"

    return pd.concat([train_metrics, val_metrics], ignore_index=True)

### Bag Of Words + Unigrams Vecotrizer

In [42]:
logistic_bow_uni_metrics = fit_pipeline_predict_evaluate(
    LogisticRegression(penalty='l2', C=0.1,max_iter=1000),
    "Logistic Regression (BoW)",
    X_train,
    y_train,
    X_val,
    y_val,
    vectorizer_type='bow',
    ngram_range=(1, 1),
)
print(logistic_bow_uni_metrics)

Fitting Logistic Regression (BoW)...
Predicting on training set...
Predicting on validation set...
Evaluating Logistic Regression (BoW) on training set...

===== Logistic Regression (BoW): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.93      0.87     10489
           1       0.89      0.73      0.80      8207

    accuracy                           0.84     18696
   macro avg       0.85      0.83      0.83     18696
weighted avg       0.85      0.84      0.84     18696

Evaluating Logistic Regression (BoW) on validation set...

===== Logistic Regression (BoW): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.89      0.81      2248
           1       0.81      0.62      0.70      1758

    accuracy                           0.77      4006
   macro avg       0.78      0.75      0.76      4006
weighted avg       0.77      0.77    

In [43]:
bow_uni_nb_metrics = fit_pipeline_predict_evaluate(
    model_class=MultinomialNB(alpha=3),
    model_name="Multinomial Naive Bayes (BoW Unigrams)",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='bow',
    ngram_range=(1, 1)
)

print(bow_uni_nb_metrics)

Fitting Multinomial Naive Bayes (BoW Unigrams)...
Predicting on training set...
Predicting on validation set...
Evaluating Multinomial Naive Bayes (BoW Unigrams) on training set...

===== Multinomial Naive Bayes (BoW Unigrams): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.93      0.90     10489
           1       0.90      0.82      0.86      8207

    accuracy                           0.88     18696
   macro avg       0.89      0.88      0.88     18696
weighted avg       0.89      0.88      0.88     18696

Evaluating Multinomial Naive Bayes (BoW Unigrams) on validation set...

===== Multinomial Naive Bayes (BoW Unigrams): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.86      0.82      2248
           1       0.80      0.70      0.75      1758

    accuracy                           0.79      4006
   macro avg       0.79   

In [44]:
bow_uni_rf_metrics = fit_pipeline_predict_evaluate(
    model_class=RandomForestClassifier(
    n_estimators=100,
    max_depth=120,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=7),
    model_name="Random Forest (BoW Unigrams)",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='bow',
    ngram_range=(1, 1)
)

print(bow_uni_rf_metrics)

Fitting Random Forest (BoW Unigrams)...
Predicting on training set...
Predicting on validation set...
Evaluating Random Forest (BoW Unigrams) on training set...

===== Random Forest (BoW Unigrams): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.94      0.80     10489
           1       0.86      0.49      0.62      8207

    accuracy                           0.74     18696
   macro avg       0.78      0.71      0.71     18696
weighted avg       0.77      0.74      0.72     18696

Evaluating Random Forest (BoW Unigrams) on validation set...

===== Random Forest (BoW Unigrams): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.93      0.79      2248
           1       0.84      0.45      0.58      1758

    accuracy                           0.72      4006
   macro avg       0.76      0.69      0.69      4006
weighted avg       0.7

In [45]:
bow_uni_mlp_metrics = fit_pipeline_predict_evaluate(
    model_class=MLPClassifier(
    hidden_layer_sizes=(32,),
    activation='relu',
    solver='adam',
    alpha=0.001,
    batch_size=64,
    learning_rate='adaptive',
    max_iter=100,
    early_stopping=True,
    random_state=42
),
    model_name="MLP",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='bow',
    ngram_range=(1, 1)
)
print(bow_uni_mlp_metrics)

Fitting MLP...
Predicting on training set...
Predicting on validation set...
Evaluating MLP on training set...

===== MLP: Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.95      0.93     10489
           1       0.94      0.88      0.91      8207

    accuracy                           0.92     18696
   macro avg       0.92      0.92      0.92     18696
weighted avg       0.92      0.92      0.92     18696

Evaluating MLP on validation set...

===== MLP: Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.85      0.82      2248
           1       0.79      0.73      0.76      1758

    accuracy                           0.80      4006
   macro avg       0.79      0.79      0.79      4006
weighted avg       0.79      0.80      0.79      4006

  Model  Accuracy  Precision    Recall  F1-score     Dataset
0   MLP  0.922657   0.935912  0

### Bag of Words + Bigrams Vecotrizer:

In [46]:
logistic_bow_bigrams_metrics = fit_pipeline_predict_evaluate(
    LogisticRegression(penalty='l2', C=0.05,max_iter=1000),
    "Logistic Regression (BoW + Bigrams)",
    X_train,
    y_train,
    X_val,
    y_val,
    vectorizer_type='bow',
    ngram_range=(1, 2)
)
print(logistic_bow_bigrams_metrics)

Fitting Logistic Regression (BoW + Bigrams)...
Predicting on training set...
Predicting on validation set...
Evaluating Logistic Regression (BoW + Bigrams) on training set...

===== Logistic Regression (BoW + Bigrams): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.95      0.88     10489
           1       0.92      0.73      0.81      8207

    accuracy                           0.85     18696
   macro avg       0.87      0.84      0.85     18696
weighted avg       0.86      0.85      0.85     18696

Evaluating Logistic Regression (BoW + Bigrams) on validation set...

===== Logistic Regression (BoW + Bigrams): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.91      0.81      2248
           1       0.83      0.56      0.67      1758

    accuracy                           0.76      4006
   macro avg       0.78      0.73      0.

In [47]:
bow_bi_nb_metrics = fit_pipeline_predict_evaluate(
    model_class=MultinomialNB(alpha=10),
    model_name="Multinomial Naive Bayes (BoW Bigrams)",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='bow',
    ngram_range=(1, 2)
)

print(bow_bi_nb_metrics)

Fitting Multinomial Naive Bayes (BoW Bigrams)...
Predicting on training set...
Predicting on validation set...
Evaluating Multinomial Naive Bayes (BoW Bigrams) on training set...

===== Multinomial Naive Bayes (BoW Bigrams): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.97      0.92     10489
           1       0.96      0.81      0.88      8207

    accuracy                           0.90     18696
   macro avg       0.91      0.89      0.90     18696
weighted avg       0.91      0.90      0.90     18696

Evaluating Multinomial Naive Bayes (BoW Bigrams) on validation set...

===== Multinomial Naive Bayes (BoW Bigrams): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.92      0.82      2248
           1       0.86      0.59      0.70      1758

    accuracy                           0.78      4006
   macro avg       0.80      0.

In [48]:
bow_bi_rf_metrics = fit_pipeline_predict_evaluate(
    model_class=RandomForestClassifier(
    n_estimators=100,
    max_depth=120,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=7),
    model_name="Random Forest (BoW Bigrams)",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='bow',
    ngram_range=(1, 2)
)

print(bow_bi_rf_metrics)

Fitting Random Forest (BoW Bigrams)...
Predicting on training set...
Predicting on validation set...
Evaluating Random Forest (BoW Bigrams) on training set...

===== Random Forest (BoW Bigrams): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.96      0.81     10489
           1       0.91      0.45      0.61      8207

    accuracy                           0.74     18696
   macro avg       0.80      0.71      0.71     18696
weighted avg       0.79      0.74      0.72     18696

Evaluating Random Forest (BoW Bigrams) on validation set...

===== Random Forest (BoW Bigrams): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.95      0.79      2248
           1       0.87      0.42      0.56      1758

    accuracy                           0.72      4006
   macro avg       0.78      0.68      0.68      4006
weighted avg       0.76    

In [49]:
bow_bi_mlp_metrics = fit_pipeline_predict_evaluate(
    model_class=MLPClassifier(
    hidden_layer_sizes=(32,),
    activation='relu',
    solver='adam',
    alpha=0.001,
    batch_size=64,
    learning_rate='adaptive',
    max_iter=100,
    early_stopping=True,
    random_state=42
),
    model_name="MLP",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='bow',
    ngram_range=(1, 2)
)
print(bow_bi_mlp_metrics)

Fitting MLP...
Predicting on training set...
Predicting on validation set...
Evaluating MLP on training set...

===== MLP: Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98     10489
           1       0.98      0.97      0.97      8207

    accuracy                           0.98     18696
   macro avg       0.98      0.98      0.98     18696
weighted avg       0.98      0.98      0.98     18696

Evaluating MLP on validation set...

===== MLP: Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.85      0.83      2248
           1       0.79      0.75      0.77      1758

    accuracy                           0.80      4006
   macro avg       0.80      0.80      0.80      4006
weighted avg       0.80      0.80      0.80      4006

  Model  Accuracy  Precision    Recall  F1-score     Dataset
0   MLP  0.978070   0.978872  0

### TF-IDF Vectorizer:

In [50]:
logistic_tfidf_metrics = fit_pipeline_predict_evaluate(
    LogisticRegression(penalty='l2', C=0.1, max_iter=1000),
    "Logistic Regression",
    X_train,
    y_train,
    X_val,
    y_val,
    vectorizer_type='tfidf',
    ngram_range=(1, 1)
)
print(logistic_tfidf_metrics)

Fitting Logistic Regression...
Predicting on training set...
Predicting on validation set...
Evaluating Logistic Regression on training set...

===== Logistic Regression: Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.97      0.80     10489
           1       0.91      0.42      0.58      8207

    accuracy                           0.73     18696
   macro avg       0.79      0.70      0.69     18696
weighted avg       0.78      0.73      0.70     18696

Evaluating Logistic Regression on validation set...

===== Logistic Regression: Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.95      0.78      2248
           1       0.86      0.40      0.55      1758

    accuracy                           0.71      4006
   macro avg       0.77      0.67      0.67      4006
weighted avg       0.75      0.71      0.68      4006

            

In [ ]:
tfidf_uni_nb_metrics = fit_pipeline_predict_evaluate(
    model_class=MultinomialNB(alpha=7),
    model_name="Multinomial Naive Bayes (TF-IDF Unigrams)",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='tfidf',
    ngram_range=(1, 1)
)

print(tfidf_uni_nb_metrics)

Fitting Multinomial Naive Bayes (TF-IDF Unigrams)...
Predicting on training set...
Predicting on validation set...
Evaluating Multinomial Naive Bayes (TF-IDF Unigrams) on training set...

===== Multinomial Naive Bayes (TF-IDF Unigrams): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.81      0.89     12532
           1       0.86      0.99      0.92     14578

    accuracy                           0.91     27110
   macro avg       0.92      0.90      0.90     27110
weighted avg       0.92      0.91      0.90     27110

Evaluating Multinomial Naive Bayes (TF-IDF Unigrams) on validation set...

===== Multinomial Naive Bayes (TF-IDF Unigrams): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.79      0.87      2685
           1       0.85      0.98      0.91      3124

    accuracy                           0.90      5809
   macro av

In [ ]:
tfidf_uni_rf_metrics = fit_pipeline_predict_evaluate(
    model_class=RandomForestClassifier(
    n_estimators=100,
    max_depth=120,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=7),
    model_name="Random Forest (TF-IDF Unigrams)",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='tfidf',
    ngram_range=(1, 1)
)

print(tfidf_uni_rf_metrics)

Fitting Random Forest (TF-IDF Unigrams)...
Predicting on training set...
Predicting on validation set...
Evaluating Random Forest (TF-IDF Unigrams) on training set...

===== Random Forest (TF-IDF Unigrams): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99     12532
           1       0.99      1.00      0.99     14578

    accuracy                           0.99     27110
   macro avg       0.99      0.99      0.99     27110
weighted avg       0.99      0.99      0.99     27110

Evaluating Random Forest (TF-IDF Unigrams) on validation set...

===== Random Forest (TF-IDF Unigrams): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97      2685
           1       0.97      0.98      0.98      3124

    accuracy                           0.97      5809
   macro avg       0.97      0.97      0.97      5809
weighte

In [ ]:
tfidf_mlp_metrics = fit_pipeline_predict_evaluate(
    model_class=MLPClassifier(
    hidden_layer_sizes=(32,),
    activation='relu',
    solver='adam',
    alpha=0.001,
    batch_size=64,
    learning_rate='adaptive',
    max_iter=100,
    early_stopping=True,
    random_state=42
),
    model_name="MLP",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='tfidf',
    ngram_range=(1, 1)
)
print(tfidf_mlp_metrics)